
# Ejercicio 7: Bases de Datos Vectoriales
### Objetivo de la práctica

Entender el concepto de Bases de Datos Vectoriales y saber utilizar las herramientas actuales




## Parte 0: Carga del Corpus

Vamos a utilizar la API de Kaggle para acceder al dataset Wikipedia Text Corpus for NLP and LLM Projects

1. Carga el corpus

In [29]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [30]:
# Set the path to the file you'd like to load
file_path = "wikipedia_text_corpus.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects",
  file_path,
)

df.head()

Using Colab cache for faster access to the 'wikipedia-text-corpus-for-nlp-and-llm-projects' dataset.


,Unnamed: 0,text
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...
1,2,Battery indicator\n\nA battery indicator (also...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...


### Parte 1: Generación de Embeddings

1. Normalizar el corpus
2. Definir una función chunk_text, y dividir los textos en chunks.
3. Generar embeddings por cada chunk

In [31]:
# Importar Librerias
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re



In [32]:
#Limpieza y normalizacion del corpus
df = df.dropna(subset=["text"]).reset_index(drop=True)

def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

,Unnamed: 0,text,text_norm
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...,Anovo Anovo (formerly A Novo) is a computer se...
1,2,Battery indicator\n\nA battery indicator (also...,Battery indicator A battery indicator (also kn...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19...","Bob Pease Robert Allen Pease (August 22, 1940Â..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...,CAVNET CAVNET was a secure military forum whic...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...,CLidar The CLidar is a scientific instrument u...


In [33]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

(   doc_id  chunk_id                                               text
 0       0         0  Anovo Anovo (formerly A Novo) is a computer se...
 1       1         0  Battery indicator A battery indicator (also kn...
 2       1         1  ad battery when in reality it indicates a prob...
 3       1         2  s that an internal standby battery needs repla...
 4       1         3  increase; in many cases the EMF remains more o...,
 79104)

In [34]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME)

# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

In [35]:
# Embeddings (N x D)
# Se debe usar normalize_embeddings=True para similitud coseno
embeddings = model.encode(
    passages,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

Batches:   0%|          | 0/4944 [00:00<?, ?it/s]

In [36]:
print(embeddings.shape, embeddings.dtype)

(79104, 768) float32


In [37]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"

query_vec = embed_query(query_text)
query_vec.shape

(1, 768)

## Parte 2: FAISS
FAISS es una librería para búsqueda por similitud eficiente y clustering de vectores densos.

1. Crea un índice en FAISS
2. Carga los embeddings
3. Realiza una búsqueda a partir de una query

In [38]:
!pip install faiss-cpu

In [39]:
import faiss

dimension = embeddings.shape[1]

index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

D, I = index.search(query_vec, 10)

## Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)
### Objetivo
Recrear el mismo flujo que con FAISS, pero usando una base vectorial con soporte nativo de metadata y filtros.

### Levantar / conectar con una instancia de Qdrant.

In [40]:
#Instalar Qdrant Client
!pip install qdrant-client

In [41]:
#Conectar a Qdrant
from qdrant_client import QdrantClient

client = QdrantClient(":memory:")

### Crear una colección con:
    - dimensión D (la de tus embeddings)
    - métrica (cosine o L2)

In [42]:
from qdrant_client.models import Distance, VectorParams

D = embeddings.shape[1]

client.create_collection(
    collection_name="documents",
    vectors_config=VectorParams(
        size=D,
        distance=Distance.COSINE
    )
)


True

### Insertar datos:
- id
- embedding
- payload (metadata: texto, título, etiquetas, etc.)

In [43]:
#Metadata
texts = [
    "Battery voltage measurement",
    "Temperature sensor calibration",
    "Power supply diagnostics"
]

titles = [
    "Battery Guide",
    "Sensor Manual",
    "Power Troubleshooting"
]

tags = [
    ["battery", "voltage"],
    ["sensor", "temperature"],
    ["power", "diagnostics"]
]


In [44]:
#Insertar en Qdrant
from qdrant_client.models import PointStruct

points = []

for i, (emb, text, title, tag) in enumerate(zip(embeddings, texts, titles, tags)):
    points.append(
        PointStruct(
            id=i,
            vector=emb.tolist(),
            payload={
                "text": text,
                "title": title,
                "tags": tag
            }
        )
    )

client.upsert(
    collection_name="documents",
    points=points
)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

### Consultar Top-k por similitud:
- query_embedding
- k

In [45]:
query_vec = embed_query(query_text)
print(query_vec.shape)  # (1, D)


(1, 768)


In [46]:
query_vector = query_vec[0].tolist()


In [47]:
!pip install -U qdrant-client

In [54]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

client = QdrantClient(":memory:")

D = embeddings.shape[1]

client.create_collection(
    collection_name="documents",
    vectors_config=VectorParams(
        size=D,
        distance=Distance.COSINE
    )
)

True

In [57]:
from qdrant_client.models import PointStruct

points = []

for i, (emb, text, title, tag) in enumerate(zip(embeddings, texts, titles, tags)):
    points.append(
        PointStruct(
            id=i,
            vector=emb.tolist(),
            payload={
                "text": text,
                "title": title,
                "tags": tag
            }
        )
    )

client.upsert(
    collection_name="documents",
    points=points
)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [58]:
query_text = "Battery measuring"
query_vec = embed_query(query_text)

query_vector = query_vec[0].tolist()

results = client.query_points(
    collection_name="documents",
    query=query_vector,
    limit=5
)

for p in results.points:
    print(p.id, p.score, p.payload["title"])


1 0.8618004140099393 Sensor Manual
2 0.8222029028893386 Power Troubleshooting
0 0.7173316768913104 Battery Guide


### top k + filtros

In [63]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

k = 5

battery_filter = Filter(
    must=[
        FieldCondition(
            key="tags",
            match=MatchValue(value="battery")
        )
    ]
)

results = client.query_points(
    collection_name="documents",
    query=query_vector,   # ✅ AQUÍ está la corrección
    limit=k,
    query_filter=battery_filter
)

for p in results.points:
    print(p.id, p.score, p.payload["title"])



0 0.7173316768913104 Battery Guide


### Funcion Qdrant_search

In [65]:
def qdrant_search(query_embedding, k=5):

    # Qdrant espera una lista, no numpy array
    query_vector = query_embedding[0].tolist()

    results = client.query_points(
        collection_name="documents",
        query=query_vector,
        limit=k
    )

    output = []

    for p in results.points:
        output.append(
            (
                p.id,
                p.score,
                p.payload.get("text"),
                {
                    "title": p.payload.get("title"),
                    "tags": p.payload.get("tags")
                }
            )
        )

    return output


In [66]:
query_text = "Battery measuring"
query_vec = embed_query(query_text)

results = qdrant_search(query_vec, k=5)

for r in results:
    print(r)


(1, 0.8618004140099393, 'Temperature sensor calibration', {'title': 'Sensor Manual', 'tags': ['sensor', 'temperature']})
(2, 0.8222029028893386, 'Power supply diagnostics', {'title': 'Power Troubleshooting', 'tags': ['power', 'diagnostics']})
(0, 0.7173316768913104, 'Battery voltage measurement', {'title': 'Battery Guide', 'tags': ['battery', 'voltage']})


### Preguntas
¿La métrica usada fue cosine o L2? ¿Por qué?
- Se utilizó la métrica Cosine o similitud coseno, ya que los embeddings fueron normalizados durante el proceso de generación. En este contexto, la similitud coseno es más adecuada porque mide la dirección del vector y no su magnitud.

¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
- El filtrado por metadata en Qdrant fue más sencillo y directo que en FAISS. Debido a que Qdrant permite almacenar metadata de forma nativa y aplicar filtros estructurados.

¿Qué pasa con el tiempo de respuesta cuando aumentas k?
- Al aumentar el valor de k, el tiempo de respuesta incrementa gradualmente, ya que el sistema debe recuperar y ordenar un mayor número de vectores similares.




## Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)

### Objetivo

Implementar el flujo de indexación + búsqueda con una base vectorial orientada a escalabilidad.

### Conectar a Milvus

In [69]:
!pip install "pymilvus[milvus_lite]"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 14.9 MB/s eta 0:00:00


In [70]:
from pymilvus import connections

connections.connect(
    alias="default",
    uri="milvus.db"   # base local persistente
)


### Crear un esquema (colección) con:
- campo id (entero o string)
- campo embedding (vector D)
- campos de metadata (p.ej., category, source, title)

In [71]:
from pymilvus import (
    FieldSchema, CollectionSchema, DataType, Collection
)

D = embeddings.shape[1]

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=D),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=200),
    FieldSchema(name="category", dtype=DataType.VARCHAR, max_length=100),
    FieldSchema(name="source", dtype=DataType.VARCHAR, max_length=100),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=1000),
]

schema = CollectionSchema(
    fields=fields,
    description="Document embeddings"
)

collection = Collection(
    name="documents_milvus",
    schema=schema
)


### Insertar N embeddings.

In [83]:
texts = df["text"].astype(str).tolist()
n = len(texts)

print("Total documentos:", n)



Total documentos: 10859


In [84]:
titles = df["text"].astype(str).str[:50].tolist()

categories = [
    "battery" if "battery" in text.lower() else "other"
    for text in texts
]

sources = ["wikipedia"] * n
ids = [str(i) for i in range(n)]


In [85]:
embeddings = model.encode(
    texts,
    convert_to_numpy=True,
    normalize_embeddings=True,
    batch_size=64,
    show_progress_bar=True
).astype("float32")

assert len(embeddings) == n
print("Embeddings generados:", embeddings.shape)




Batches:   0%|          | 0/170 [00:00<?, ?it/s]

Embeddings generados: (10859, 768)


In [86]:
from pymilvus import utility

if utility.has_collection("wiki_docs"):
    utility.drop_collection("wiki_docs")


In [87]:
from pymilvus import CollectionSchema, FieldSchema, DataType

dim = embeddings.shape[1]

fields = [
    FieldSchema(name="id", dtype=DataType.VARCHAR, max_length=32, is_primary=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=dim),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=128),
    FieldSchema(name="category", dtype=DataType.VARCHAR, max_length=64),
    FieldSchema(name="source", dtype=DataType.VARCHAR, max_length=64),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=512),
]

schema = CollectionSchema(fields, description="Wikipedia corpus")

collection = Collection("wiki_docs", schema)


In [93]:
MAX_TEXT_LEN = 512
texts = [t[:MAX_TEXT_LEN] for t in texts]
titles = [t[:128] for t in titles]


In [94]:
collection.insert([
    ids,
    embeddings.tolist(),
    titles,
    categories,
    sources,
    texts
])

collection.flush()
print("Corpus insertado en Milvus")



Corpus insertado en Milvus


### Crear/seleccionar un índice ANN (ej. HNSW o IVF).

In [95]:
collection.create_index(
    field_name="embedding",
    index_params={
        "index_type": "FLAT",
        "metric_type": "COSINE",
        "params": {}
    }
)

collection.load()



### Ejecutar consultas Top-k y recuperar textos asociados.

In [96]:
query_text = "Battery measuring"
query_vec = embed_query(query_text)

results = collection.search(
    data=query_vec.tolist(),
    anns_field="embedding",
    param={"metric_type": "COSINE"},
    limit=5,
    output_fields=["title", "category", "source", "text"]
)

for hit in results[0]:
    print(hit.id, hit.score, hit.entity.get("title"))



1391 0.8678303956985474 Battery tester

A battery tester is an electronic 
1 0.8469709157943726 Battery indicator

A battery indicator (also known
1445 0.833725094795227 Current sense monitor

A Current Sense Monitor is 


In [99]:
collection.release()
collection.drop_index()


In [100]:
collection.create_index(
    field_name="embedding",
    index_params={
        "index_type": "IVF_FLAT",
        "metric_type": "COSINE",
        "params": {
            "nlist": 1024
        }
    }
)


Status(code=0, message=)

In [102]:
collection.load()
print("Índice IVF_FLAT creado")




Índice IVF_FLAT creado


In [103]:
search_params = {
    "metric_type": "COSINE",
    "params": {
        "nprobe": 16
    }
}

results_ann = collection.search(
    data=query_vec.tolist(),
    anns_field="embedding",
    param=search_params,
    limit=5,
    output_fields=["title", "category", "source", "text"]
)


In [105]:
#Resultados ANN
for hit in results_ann[0]:
    print(hit.id, hit.score, hit.entity.get("title"))


1391 0.8678303956985474 Battery tester

A battery tester is an electronic 
1 0.8469709157943726 Battery indicator

A battery indicator (also known
1445 0.833725094795227 Current sense monitor

A Current Sense Monitor is 


### Configuracion EXACTA y ANN

In [106]:
#Comparacion
exact_ids = {hit.id for hit in results[0]}
ann_ids = {hit.id for hit in results_ann[0]}

print("IDs en común:", exact_ids & ann_ids)

IDs en común: {'1', '1391', '1445'}


### Comparacion de tiempos

In [107]:
#Medicion de tiempos
import time

# EXACTO
start = time.time()
collection.search(
    data=query_vec.tolist(),
    anns_field="embedding",
    param={"metric_type": "COSINE"},
    limit=5
)
t_exact = time.time() - start

# ANN
start = time.time()
collection.search(
    data=query_vec.tolist(),
    anns_field="embedding",
    param=search_params,
    limit=5
)
t_ann = time.time() - start

print("Tiempo exacto:", t_exact)
print("Tiempo ANN:", t_ann)



Tiempo exacto: 0.0175628662109375
Tiempo ANN: 0.012784719467163086


### Funcion weaviate_search

In [108]:
import time

def milvus_search(query_embedding, k):

    search_params = {
        "metric_type": "COSINE",
        "params": {
            "nprobe": 16
        }
    }

    start = time.time()
    results = collection.search(
        data=query_embedding.tolist(),
        anns_field="embedding",
        param=search_params,
        limit=k,
        output_fields=["title", "category", "source", "text"]
    )
    elapsed_time = time.time() - start

    parsed_results = []
    for hit in results[0]:
        parsed_results.append({
            "id": hit.id,
            "score": hit.score,
            "title": hit.entity.get("title"),
            "category": hit.entity.get("category"),
            "source": hit.entity.get("source"),
            "text": hit.entity.get("text")
        })

    return elapsed_time, parsed_results


In [110]:
query_text = "Battery measuring"
query_vec = embed_query(query_text)


In [113]:
#k = 5

time_k5, results_k5 = milvus_search(query_vec, k=5)

print("k = 5")
print("Tiempo:", time_k5)
for r in results_k5:
    print(r["id"], r["score"], r["title"])


k = 5
Tiempo: 0.016744136810302734
1391 0.8678303956985474 Battery tester

A battery tester is an electronic 
1 0.8469709157943726 Battery indicator

A battery indicator (also known
1445 0.833725094795227 Current sense monitor

A Current Sense Monitor is 


In [114]:
#k = 20

time_k20, results_k20 = milvus_search(query_vec, k=20)

print("\nk = 20")
print("Tiempo:", time_k20)
for r in results_k20[:5]:  # mostramos solo los primeros 5
    print(r["id"], r["score"], r["title"])



k = 20
Tiempo: 0.020487070083618164
1391 0.8678303956985474 Battery tester

A battery tester is an electronic 
1 0.8469709157943726 Battery indicator

A battery indicator (also known
1445 0.833725094795227 Current sense monitor

A Current Sense Monitor is 
9888 0.8266297578811646 Float voltage

Float voltage is the voltage at whi
2076 0.8260664939880371 Voltmeter

A voltmeter is an instrument used for m


### Preguntas
¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?
- En el enfoque de “schema + objetos”, los datos se representan como entidades auto-contenidas.
- En contraste, el enfoque de “tabla + filas” se basa en un esquema estricto y tipado, donde cada fila representa un registro y todas las columnas deben estar definidas previamente.

¿Cómo describirías el trade-off de complejidad vs expresividad?

- Las bases vectoriales orientadas a objetos ofrecen mayor expresividad con menor complejidad inicial.
- Por otro lado, los sistemas basados en esquemas estrictos incrementan la complejidad inicial.

## Parte 6 — Vector Store #4: Chroma (prototipado rápido)

###Objetivo

Implementar la misma idea de indexación y búsqueda semántica con una herramienta ligera de prototipado.

### Importaciones e Instalaciones

In [117]:
!pip install chromadb


In [118]:
import chromadb
from chromadb.config import Settings

### Crear una colección

In [119]:
client = chromadb.Client(
    Settings(
        persist_directory="./chroma_db",
        anonymized_telemetry=False
    )
)

collection = client.get_or_create_collection(
    name="wiki_docs"
)


In [120]:
#Preparar los datos
ids = [str(i) for i in range(len(embeddings))]
documents = texts  # textos truncados a 512
metadatas = [
    {
        "title": titles[i],
        "category": categories[i],
        "source": sources[i]
    }
    for i in range(len(texts))
]


### Insertar en Chroma

In [122]:
def batch_insert_chroma(
    collection,
    ids,
    embeddings,
    documents,
    metadatas,
    batch_size=2000
):
    for i in range(0, len(ids), batch_size):
        collection.add(
            ids=ids[i:i+batch_size],
            embeddings=embeddings[i:i+batch_size].tolist(),
            documents=documents[i:i+batch_size],
            metadatas=metadatas[i:i+batch_size]
        )

    print("Corpus insertado en Chroma en batches")



In [123]:
batch_insert_chroma(
    collection,
    ids,
    embeddings,
    documents,
    metadatas
)


Corpus insertado en Chroma en batches


### Consultas top K

In [124]:
query_text = "Battery measuring"
query_vec = embed_query(query_text)


In [125]:
results = collection.query(
    query_embeddings=query_vec.tolist(),
    n_results=5
)


In [126]:
#Mostra resultados
for i in range(len(results["ids"][0])):
    print(
        results["ids"][0][i],
        results["distances"][0][i],
        results["metadatas"][0][i]["title"]
    )



1391 0.2643393278121948 Battery tester

A battery tester is an electronic 
1 0.3060583472251892 Battery indicator

A battery indicator (also known
1445 0.33254992961883545 Current sense monitor

A Current Sense Monitor is 
9888 0.34674060344696045 Float voltage

Float voltage is the voltage at whi
2076 0.3478671908378601 Voltmeter

A voltmeter is an instrument used for m


### Funcion chroma_search

In [127]:
def chroma_search(query_embedding, k):
    results = collection.query(
        query_embeddings=query_embedding.tolist(),
        n_results=k
    )

    parsed = []
    for i in range(len(results["ids"][0])):
        parsed.append({
            "id": results["ids"][0][i],
            "score": results["distances"][0][i],
            "text": results["documents"][0][i],
            "metadata": results["metadatas"][0][i]
        })

    return parsed


### Consulta k = 5

In [128]:
res = chroma_search(query_vec, k=5)

for r in res:
    print(r["id"], r["score"], r["metadata"]["title"])


1391 0.2643393278121948 Battery tester

A battery tester is an electronic 
1 0.3060583472251892 Battery indicator

A battery indicator (also known
1445 0.33254992961883545 Current sense monitor

A Current Sense Monitor is 
9888 0.34674060344696045 Float voltage

Float voltage is the voltage at whi
2076 0.3478671908378601 Voltmeter

A voltmeter is an instrument used for m


### Preguntas

¿Qué tan fácil fue implementar todo comparado con Qdrant y Milvus?
- Chroma permite crear colecciones, insertar embeddings y ejecutar búsquedas Top-k con muy pocas líneas de código y sin requerir configuración explícita de esquemas, índices ANN o servicios externos.

¿Qué limitaciones ves para un sistema en producción?

- Escalabilidad limitada: Chroma no está diseñado para manejar volúmenes masivos de datos ni cargas concurrentes elevadas.

- Persistencia y despliegue: Su modelo de persistencia es más básico y no está orientado a despliegues distribuidos.

- Filtros y consultas complejas: El soporte para filtrado por metadata y consultas avanzadas es más limitado en comparación con Qdrant.



## Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)
### Objetivo
Guardar embeddings en una tabla y ejecutar una consulta SQL de similitud.

### Conectar a una base PostgreSQL con pgvector habilitado.

Usaremos una BD en linea SUPABASE

In [130]:
#Instalar el driver
!pip install python-dotenv psycopg2

In [140]:
import psycopg2
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

PASSWORD = "Shinonuma1@"

# Connect to the database
try:
    connection = psycopg2.connect(
        user="postgres",
        password=PASSWORD,
        host="db.shyqiwvsshfmtakhqajo.supabase.co",
        port=5432,
        dbname="postgres"
    )
    print("Connection successful!")

    # Create a cursor to execute SQL queries
    cursor = connection.cursor()

    # Example query
    cursor.execute("SELECT NOW();")
    result = cursor.fetchone()
    print("Current Time:", result)

    # Close the cursor and connection
    cursor.close()
    connection.close()
    print("Connection closed.")

except Exception as e:
    print(f"Failed to connect: {e}")


Failed to connect: connection to server at "db.shyqiwvsshfmtakhqajo.supabase.co" (2600:1f18:2e13:9d3b:b1de:dfdc:c1e3:88d6), port 5432 failed: Network is unreachable
	Is the server running on that host and accepting TCP/IP connections?



### Creacion de la TABLA

In [141]:
# Habilitar pgvector
cursor = connection.cursor()

cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")
connection.commit()

print("pgvector habilitado")


NameError: name 'connection' is not defined

In [132]:
D = embeddings.shape[1]
print("Dimensión:", D)

Dimensión: 768


In [133]:
cursor.execute(f"""
CREATE TABLE IF NOT EXISTS documents (
    id SERIAL PRIMARY KEY,
    text TEXT,
    title TEXT,
    category TEXT,
    source TEXT,
    embedding VECTOR({D})
);
""")
conn.commit()
print("Tabla documents creada")


NameError: name 'cursor' is not defined